<h1 style="text-align: center;">Final Project </h1>

$\bullet$ __ We first download load the data __

In [2]:
import pandas as pd
import os

df_reviews = pd.DataFrame()
file_list = os.listdir("../yelp_dataset/reviews_filter.json")

for file in file_list:
    # loading the reviews into a dataframe
    df = pd.read_json("../yelp_dataset/reviews_filter.json/" + file, lines=True) #specify that json has multiple lines
    df_reviews = pd.concat([df_reviews, df])

# removing rows containing NaN value and reaffecting index to the df
df_reviews = df_reviews.dropna() 
df_reviews = df_reviews.reset_index(drop=True)

print(len(df_reviews))

474432


In [3]:
# loading the business into a dataframe
df_business = pd.DataFrame()
file_list = os.listdir("../yelp_dataset/open_business.json")
    
for file in file_list:
    # loading business into a dataframe
    df = pd.read_json("../yelp_dataset/open_business.json/" + file, lines=True) #specify that json has multiple lines
    df_business = pd.concat([df_business, df])

# removing rows containing NaN value and reaffecting index to the df
df_business = df_business.dropna() 
df_business = df_business.reset_index(drop=True)

print(len(df_business))

108521


In [4]:
print("Reviews keys : ", list(df_reviews.keys()))
print("Business keys : ", list(df_business.keys()))

Reviews keys :  ['business_id', 'date', 'review_id', 'stars', 'text', 'user_id']
Business keys :  ['address', 'attributes', 'business_id', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood', 'postal_code', 'review_count', 'stars', 'state']


<hr>

$\bullet$ __ The idea is now to filter the business dataframe in order to select only the business that have reviews__

In [5]:
list_business = list(df_reviews.business_id.unique())
print(len(list_business))

# selecting only the wanted business
df_business = df_business[df_business['business_id'].isin(list_business) == True]
df_business = df_business.reset_index(drop=True)
print(len(df_business))

# strange thing : the two following lists do not have the same length
# print(len(list_business) - len(df_business))
# print(list(set(list_business) - set(list(df_business.business_id.values))))

8191
7799


<hr>

# Analyze comment 

In [6]:
import nltk
from nltk import word_tokenize
labMT = pd.read_csv('data/s001.txt', sep="\t")

In [7]:
# function computing the sentimental value of a text
def computeSentiment(text):
    happiness_average, text = 0, word_tokenize(text)
    words = {word: text.count(word) for word in set(text)}
    
    for word in words:
        val_happiness = labMT[labMT["word"] == word].happiness_average.values
        # checking if the word is in labMT or not
        if len(val_happiness)>0:
            happiness_average += float(val_happiness[0]) * words[word] # taking into account the nb of occurences of the word
            
    return happiness_average / len(text)   

$\bullet$ __Now creating a dictionnary where keys are business id and value a list of all the comments related to this business. In order to compute the rating of each business. And another one where keys are business id and values the sentiment value__

In [ ]:
dic_reviews = {business: [] for business in list_business}

for i in range(len(df_reviews)):
    comment = df_reviews.loc[i].text
    business = df_reviews.loc[i].business_id
    
    dic_reviews[business].append(comment)

$\bullet$ __ Use the first cell to create a json file containing as a key the business id and as value the rate of the busines. Use the second one to load the data from the file.__

In [ ]:
import json
from statistics import mean
dic_rate = {}

for elt in list(dic_reviews.keys()):
    dic_rate[elt] = mean([computeSentiment(text) for text in dic_reviews[elt]])

with open("dic_rate.json",'w') as file:
    json.dump(dic_rate, file, indent=4)

In [8]:
import json

with open("dic_rate.json",'r') as file:
    dic_rate = json.load(file)

<hr>

# Showing positions on a map

In [9]:
# filtering business in order to get business in a small area
df_business["latitude"] = pd.to_numeric(df_business["latitude"]) # changing type of the column to float
df_business["longitude"] = pd.to_numeric(df_business["longitude"])

# df_business = df_business[(df_business["longitude"] > -90) & (df_business['longitude'] < -85) &(df_business['latitude'] < 42)]
# df_business = df_business.reset_index(drop=True)
# print(len(df_business))

Use either one of the following locations :
<li> First one gets all business locations </li>
<li> Second one gets only the locations of the business which sentimental mark is computed</li>

In [10]:
# creating a list containing all localisations of business
locations = []
for i in range(len(df_business)):
    locations.append((df_business.loc[i].latitude, df_business.loc[i].longitude))

In [11]:
# creating a list containing the localisations of all the analyzed business
locations = {}
for business in list(dic_rate.keys()):
    try:
        locations[business] =((df_business[df_business['business_id'] == business].latitude.values[0], df_business[df_business['business_id'] == business].longitude.values[0]))
    except IndexError:
        pass

In [12]:
import statistics

# here we compute the color of each marker (business) regarding their final mark
max_rate = max(dic_rate.values())
med_rate = statistics.median(dic_rate.values())
min_rate = min(dic_rate.values())

def aggregateColors(rate):
    if rate <= med_rate:
        x = int(255*(rate - min_rate)/(med_rate - min_rate))
        return (255, x, 0, 0.8)
    else:
        x = int(255*(rate - med_rate)/(max_rate - med_rate))
        return (255 - x, 255, 0, 0.8)

colors = [aggregateColors(dic_rate[elt]) for elt in list(locations.keys())]
print(len(locations))
print(len(colors))

7799
7799


In [ ]:
import gmaps
gmaps.configure(api_key='API_KEY')

place_focus = (33.448376, -112.074036) #focusing on phenix
fig = gmaps.figure(
        center = place_focus,
        zoom_level = 10)
symbols = gmaps.symbol_layer(
        list(locations.values()), 
        fill_color=colors, 
        stroke_color=colors)
fig.add_layer(symbols)

# hm = gmaps.heatmap_layer(locations, weights=rate, max_intensity=max(rate), point_radius=5.0)
# fig.add_layer(hm)

# fig

# Trying to operate business categories

$\bullet$ __Changing the type of the column categories to a list__

In [13]:
df_business.categories = df_business.categories.astype(list)
df_business.categories = df_business['categories'].map(lambda a: a.split(",")) # spliting each element of row into a list

$\bullet$ __Finding which categories are the most used__

In [14]:
# function returning a dictionnary with key string and value the nb of occurence
def countingOccurencesListOfLists(list_of_lists):
    flattened = [val.lstrip() for sublist in list_of_lists for val in sublist] # flattening the list of lists
    dic = {key : 0 for key in flattened}
    for row in list_of_lists:
        for elt in row:
            dic[elt.lstrip()] += 1
            
    return dic

In [15]:
import operator

# sorting the tuples according to their nb of occurences
a = countingOccurencesListOfLists(list(df_business.categories.values))
a = sorted(a.items(), key=lambda item: (item[1], item[0]), reverse=True)

$\bullet$ __Filter the business by a list of categories__

In [31]:
filteringItems = [elt[0] for elt in a[133:143]]

def filterDfByListElement(filteringList, df, column):
    indexToDrop = []
    for i in range(len(df)):
        categories = df.loc[i][column]
        if len(set(categories) & set(filteringList)) == 0:
            indexToDrop.append(i)
            
    df = df.drop(indexToDrop)
    df = df.reset_index(drop=True)
    return df

# Network of elite users with friends

In [83]:
import pandas as pd
import os

df_elite = pd.DataFrame()
file_list = os.listdir("../yelp_dataset/elite_users_with_friends.json")

for file in file_list:
    # loading the reviews into a dataframe
    df = pd.read_json("../yelp_dataset/elite_users_with_friends.json/" + file, lines=True) #specify that json has multiple lines
    df_elite = df_elite.append(df, ignore_index=True)
    
df_elite = df_elite.dropna() 
df_elite = df_elite.reset_index(drop=True)

$\bullet$ __Setting 'friend' and 'elite' columns as list__

In [86]:
df_elite.elite = df_elite.elite.astype(list)
df_elite.elite = df_elite['elite'].map(lambda a: a.split(",")) # spliting each element of row into a list

In [102]:
elite_2010 = filterDfByListElement(['2008'], df_elite, 'elite')

In [123]:
import networkx as nx

# G = nx.DiGraph()

# for i in range(len(elite_2010)):
#     print(i)
#     G.add_node(df_elite['user_id'][i])
#     friends_list = df_elite['friends'][i]
#     for friend in friends_list:
#         if list(df_elite['user_id']).count(friend) > 0:
#             G.add_node(friend)
#             G.add_edge(df_elite['user_id'][i], friend)
    
    

[4, 4]


## Getting Elite users with Elite friends

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import os

In [6]:
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(spark);

spark_df_users = spark.read.json("../yelp_dataset/yelp_academic_dataset_user.json")

Exception: Java gateway process exited before sending its port number

In [ ]:
spark_df_users = spark_df_users.where("friends != 'None' AND elite != 'None'")
spark_df_users = spark_df_users.drop("average_stars", "compliment_cool", "compliment_cute", "compliment_funny",
                                     "compliment_hot", "compliment_list", "compliment_more", "compliment_note",
                                     "compliment_photos", "compliment_plain", "compliment_profile",
                                     "compliment_writer", "cool", "funny", "useful", "yelping_since")

spark_df_users = spark_df_users.withColumn("friends", split(col("friends"), "\\|"))